In [1]:
import pandas as pd
import numpy as np
import os
import re
import random

In [2]:
df = pd.read_csv(r"C:\Users\NEELAM SUSHMITHA\OneDrive\Documents\RAMANASOFT INTERNSHIP\IMDB Dataset.csv",nrows=1000)

In [3]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [4]:
df['review'] = df['review'].str.lower()

In [5]:
import re  #Regular Expressions
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [6]:
df['review'] = df['review'].apply(remove_html_tags)

In [7]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www.\.\S+')
    return pattern.sub(r'', text)

In [8]:
df['review'] = df['review'].apply(remove_url)

In [9]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
exclude = string.punctuation

In [11]:
def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

In [12]:
df['review'] = df['review'].apply(remove_punc1)

In [13]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

In [14]:
from nltk.tokenize import sent_tokenize

In [15]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"review"]))
    df.at[i,"sentences"]=l1

In [16]:
!pip install pywsd

In [17]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\NEELAM SUSHMITHA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [18]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 4.662323236465454 secs.


In [19]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [20]:
df["words_sentences"] = "default"

In [21]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])

### 🔡 Vectorization :
Converting text (words/sentences) into numbers (vectors)
->Because machine learning models can't understand text — they only understand numbers(vectors).  
->Each number represents word frequency or importance. This is what vectorization does.


| Types of Text Vectorization Method                         |                      Description              |        Example Use       |
| ----------------------------------------------------------- | --------------------------------------------------------- | ----------------- |
| 1. **Bag of Words (BoW)**                                   | Count how many times each word appears                    | `CountVectorizer` |
| 2. **TF-IDF (Term Frequency – Inverse Document Frequency)** | Weight words by how important they are across documents   | `TfidfVectorizer` |
| 3. **Word Embeddings**                                      | Dense vector representation using meaning/context         | Word2Vec, GloVe   |
| 4. **Pretrained Models (Transformers)**                     | Use models like BERT to generate context-aware embeddings | Hugging Face BERT |


# Count Vectorization

->Without vectorization, you can’t do sentiment analysis, spam detection, chatbot NLP, etc.
->It’s the bridge between text and ML models.

In [22]:
from sklearn.feature_extraction.text import  CountVectorizer

In [23]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

CountVectorizer : converts text to Bag of Words format

max_features=1000 : keep only top 1000 most frequent words

stop_words='english' : removes common words like "the", "is", "and"

fit_transform() : learns vocabulary & transforms text into numbers

tf becomes a sparse matrix with shape (n_samples, 1000) where each row is a document and each column is a word.

In [24]:
df_x = pd.DataFrame(tf.toarray(), columns=tf_vectorizer.get_feature_names_out())

✅ Converts the matrix into a pandas DataFrame for modeling.

In [25]:
df_x.head()

,10,100,12,20,30,60,70,80,90,ability,...,yeah,year,yes,york,youd,youll,young,youre,youve,zombie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [26]:
df_x.shape

(1000, 1000)

In [27]:
df_y=df["sentiment"]

In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
le=LabelEncoder()

In [30]:
df_y_1=pd.DataFrame(df_y)

In [31]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [32]:
df_y_enc.columns

Index(['sentiment'], dtype='object')

In [33]:
df_y_enc.head(5)

,sentiment
0,1
1,1
2,1
3,0
4,1


### 🔰 Classical Machine Learning Models (after vectorization)

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
GBC=GradientBoostingClassifier(n_estimators=3000)

In [36]:
gb_c=GBC.fit(df_x,df_y_enc)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
GBC.score(df_x,df_y_enc)

1.0

note:✅ it’s not okay to only use .fit() and .score() on the same data (df_x and df_y_enc) without splitting into training and testing sets.

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y_enc, test_size=0.2, random_state=42)

# SVM 
It is a supervised machine learning algorithm used for classification and regression tasks. It is most commonly used for binary classification problems.

SVM finds the optimal hyperplane that best separates the data points of different classes.

In [85]:
#1. Support Vector Machine (SVM)
from sklearn.svm import SVC

svm_model = SVC(max_iter=1000)
svm_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(max_iter=1000)

In [40]:
svm_score = svm_model.score(X_test, y_test)
print("SVM Accuracy:", svm_score)

SVM Accuracy: 0.79


In [41]:
#  2. Logistic Regression
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [42]:
log_score = log_model.score(X_test, y_test)
print("Logistic Regression Accuracy:", log_score)

Logistic Regression Accuracy: 0.815


In [43]:
#  3. AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier

ada_model = AdaBoostClassifier(n_estimators=300)
ada_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(n_estimators=300)

In [44]:
ada_score = ada_model.score(X_test, y_test)
print("AdaBoost Accuracy:", ada_score)

AdaBoost Accuracy: 0.8


In [45]:
# 4. Naive Bayes
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [46]:
nb_score = nb_model.score(X_test, y_test)
print("Naive Bayes Accuracy:", nb_score)

Naive Bayes Accuracy: 0.825


In [47]:
# 5. Random Forest Classifier 
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200)
rf_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=200)

In [48]:
rf_score = rf_model.score(X_test, y_test)
print("Random Forest Accuracy:", rf_score)

Random Forest Accuracy: 0.82


### 🔁 More Classical Models:

In [88]:
# 1.K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [89]:
print("KNN Accuracy:", knn.score(X_test, y_test))

KNN Accuracy: 0.72


In [90]:
# 2.ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ExtraTreesClassifier()

In [91]:
print("ExtraTrees Accuracy:", etc.score(X_test, y_test))

ExtraTrees Accuracy: 0.78


In [92]:
# 3.XGBoost (Extreme Gradient Boosting) — very powerful
import xgboost as xgb
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:52:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [93]:
print("XGBoost Accuracy:", xgb_model.score(X_test, y_test))

XGBoost Accuracy: 0.765


# tfidf_vectorizer

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
df1=df
no_features = 1000
tfidf_vectorizer = TfidfVectorizer(max_features=no_features,stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.review)

In [58]:
df_x=pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [94]:
df_x.head()

,10,100,12,20,70s,80s,ability,able,absolutely,accent,...,years,yes,york,youd,youll,young,youre,youve,zombie,zombies
0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.09788,0.000000,0.000000,0.0,0.00000,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.119612,0.0,0.000000,0.0,0.00000,0.127912,0.000000,0.0,0.00000,0.0
3,0.11988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.126866,0.0,0.17976,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.144635,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0


In [59]:
df_y=df["sentiment"]

In [60]:
from sklearn.preprocessing import LabelEncoder

In [61]:
le=LabelEncoder()

In [62]:
df_y_1=pd.DataFrame(df_y)

In [63]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [64]:
df_y_enc.columns

Index(['sentiment'], dtype='object')

In [65]:
df_y_enc.head(5)

,sentiment
0,1
1,1
2,1
3,0
4,1


In [66]:
from sklearn.ensemble import GradientBoostingClassifier

In [67]:
GBC=GradientBoostingClassifier(n_estimators=300)

In [68]:
gb_c=GBC.fit(df_x,df_y_enc)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
GBC.score(df_x,df_y_enc)

1.0

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y_enc, test_size=0.2, random_state=42)

In [81]:
#1. Support Vector Machine (SVM)
from sklearn.svm import SVC

svm_model = SVC(max_iter=1000)
svm_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(max_iter=1000)

In [82]:
svm_score = svm_model.score(X_test, y_test)
print("SVM Accuracy:", svm_score)

SVM Accuracy: 0.815


In [73]:
#  2. Logistic Regression
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [74]:
log_score = log_model.score(X_test, y_test)
print("Logistic Regression Accuracy:", log_score)

Logistic Regression Accuracy: 0.82


In [75]:
#  3. AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier

ada_model = AdaBoostClassifier(n_estimators=300)
ada_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(n_estimators=300)

In [76]:
ada_score = ada_model.score(X_test, y_test)
print("AdaBoost Accuracy:", ada_score)

AdaBoost Accuracy: 0.75


In [84]:
# 4. Naive Bayes
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [78]:
nb_score = nb_model.score(X_test, y_test)
print("Naive Bayes Accuracy:", nb_score)

Naive Bayes Accuracy: 0.82


In [79]:
# 5. Random Forest Classifier 
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200)
rf_model.fit(X_train, y_train)

C:\Users\NEELAM SUSHMITHA\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=200)

In [80]:
rf_score = rf_model.score(X_test, y_test)
print("Random Forest Accuracy:", rf_score)

Random Forest Accuracy: 0.805
